In [23]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.graph_objects import Figure

In [24]:
mortgage_amount_nis = 1200000
num_of_years = 15
prime_interest_rate = 6

In [26]:
def keren_shava(mortgage_amount_nis, years, annual_interest_rate):
    monthly_interest_rate = (annual_interest_rate / 100) / 12
    
    total_months = years * 12
    payback_per_month = []
    remaining_loan_principal = mortgage_amount_nis
    const_payback_per_month = mortgage_amount_nis / total_months
    for i in range(total_months):
        payback_per_month.append(const_payback_per_month + monthly_interest_rate * remaining_loan_principal)
        remaining_loan_principal -= const_payback_per_month
    
    return payback_per_month

In [28]:
def shpitzer_payment(mortgage_amount_nis, years, annual_interest_rate):
    """
    Calculate the monthly payment for a mortgage using the Shpitzer method.

    Parameters:
    mortgage_amount_nis (float): Total mortgage amount in NIS.
    years (int): Number of years for the mortgage.
    annual_interest_rate (float): Annual interest rate as a percentage (e.g., 3.5 for 3.5%).

    Returns:
    float: Monthly payment amount in NIS.
    """
    # Convert annual interest rate to monthly interest rate (in decimal form)
    monthly_interest_rate = (annual_interest_rate / 100) / 12
    
    # Total number of months
    total_months = years * 12
    
    # Apply the annuity formula to calculate the monthly payment
    if monthly_interest_rate > 0:
        monthly_payment = (mortgage_amount_nis * monthly_interest_rate) / \
                          (1 - (1 + monthly_interest_rate) ** -total_months)
    else:
        # If the interest rate is 0, divide the mortgage amount by total months
        monthly_payment = mortgage_amount_nis / total_months
    
    return monthly_payment

In [29]:
path_file = r"C:\Users\eyall\Desktop\mortgage_eyal_ortal_levi\mortgage_israel_bank_info.xlsx"

banks_info = pd.read_excel(path_file)

In [30]:
banks_info

,Bank,loan_type,Weight,Interest Rate
0,Mizrachi,Constant interest rate and not index-linked,39.0,4.77
1,Mizrachi,change interest rate and not index-linked - prime,3.0,6.08
2,Mizrachi,change interest rate and not index-linked - no...,43.0,4.98
3,Mizrachi,change interest rate and index-linked,10.0,3.55
4,Mizrachi,Constant interest rate and index-linked,5.0,3.63
5,Mizrachi,IRR,NaN,5.75
6,Leomi,Constant interest rate and not index-linked,26.0,4.83
7,Leomi,change interest rate and not index-linked - prime,11.0,5.55
8,Leomi,change interest rate and not index-linked - no...,42.0,4.96
9,Leomi,change interest rate and index-linked,12.0,3.26


In [31]:
banks_info.columns

Index(['Bank', 'loan_type', 'Weight', 'Interest Rate'], dtype='object')

In [35]:
unique_bank_names = banks_info["Bank"].unique()
unique_loan_type  = banks_info["loan_type"].unique()
print(unique_loan_type)

['Constant interest rate and not index-linked'
 'change interest rate and not index-linked - prime'
 'change interest rate and not index-linked - not prime'
 'change interest rate and index-linked'
 'Constant interest rate and  index-linked' 'IRR']


In [36]:

not_index_linked_types = unique_loan_type[np.char.find(unique_loan_type.astype(str), 'not index-linked') != -1]
very_low_risk_type = unique_loan_type[np.char.find(unique_loan_type.astype(str), 'Constant interest rate and not index-linked') != -1]
all_types = unique_loan_type
combination_dict = {"low_risk": very_low_risk_type, "middle_risk": not_index_linked_types, "all": all_types}

In [37]:
def calc_total_payback_and_payback_rate_for_const_inter_not_index_linked(mortgage_amount_nis, years, annual_interest_rate):
    total_months = years * 12
    monthly_payment_shpizer_num = shpitzer_payment(mortgage_amount_nis, years, annual_interest_rate)
    monthly_payment_keren_shava_list = keren_shava(mortgage_amount_nis, years, annual_interest_rate)
    return {"shpizer": {"total_payback": monthly_payment_shpizer_num * total_months, "payback_rate": monthly_payment_shpizer_num * total_months / mortgage_amount_nis}, \
             "keren_shava": {"total_payback": sum(monthly_payment_keren_shava_list), "payback_rate": sum(monthly_payment_keren_shava_list) / mortgage_amount_nis}}

In [38]:
def calc_total_payback_and_payback_rate_per_type(loan_type, mortgage_amount_nis, years, annual_interest_rate, payback_method):
    match loan_type:
        case 'Constant interest rate and not index-linked':
            dict_payback_info = calc_total_payback_and_payback_rate_for_const_inter_not_index_linked( mortgage_amount_nis, years, annual_interest_rate)
            return dict_payback_info[payback_method]

In [39]:
traces = []
for risk_level, mix_loan_types in combination_dict.items():
    if risk_level != "low_risk": 
        continue  # for now
    
    bank_names_list = []
    shpizer_payback_rate = []
    keren_shava_playback_rate = []

    # Iterate through unique bank names
    for bank_name in unique_bank_names: 
        bank_names_list.append(bank_name)
        bank_data = banks_info.loc[banks_info['Bank'] == bank_name]
        
        for single_loan_type in mix_loan_types:
            interest_rate = bank_data.loc[bank_data["loan_type"] == single_loan_type]["Interest Rate"].values[0]
            
            dict_shpizer = calc_total_payback_and_payback_rate_per_type(
                loan_type=single_loan_type, 
                mortgage_amount_nis=mortgage_amount_nis,
                years=num_of_years, 
                annual_interest_rate=interest_rate, 
                payback_method="shpizer"
            )
            
            dict_keren_shava = calc_total_payback_and_payback_rate_per_type(
                loan_type = single_loan_type, 
                mortgage_amount_nis=mortgage_amount_nis,               
                years=num_of_years, 
                annual_interest_rate = interest_rate, 
                payback_method="keren_shava"
            )
            
            shpizer_payback_rate.append(dict_shpizer["payback_rate"])
            keren_shava_playback_rate.append(dict_keren_shava["payback_rate"])
    # Create a trace for the current risk level
    trace = go.Scatter(
        x=bank_names_list,    
        y=shpizer_payback_rate,         
        mode='lines+markers',    
        name=f'Payback Rate - {risk_level}- Shpizer Method'  # Ensure unique trace names
    )
    traces.append(trace)
    
    trace = go.Scatter(
        x=bank_names_list,    
        y=keren_shava_playback_rate,         
        mode='lines+markers',    
        name=f'Payback Rate - {risk_level} - Keren-Shava Method'  # Ensure unique trace names
    )
    traces.append(trace)

# Create the layout for the figure
layout = go.Layout(
    title="Shpizer Payback Rate by Bank",
    xaxis_title="Bank",
    yaxis_title="Shpizer Payback Rate",
    hovermode='closest',
    showlegend=True,  # Explicitly show the legend
    legend=dict(
        orientation='v',  # Vertical orientation
        x=0,  # Position on the left
        xanchor='left',  # Anchor to the left
        y=1,  # Position at the top
        yanchor='top'  # Anchor to the top
    )
)

# Create the figure once after processing all banks
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()

# Optionally, save the figure to an HTML file
# fig.write_html("figure.html")
# Open the HTML file in a browser
# import webbrowser
# import os
# file_path = os.path.abspath("figure.html")
# webbrowser.open(f"file://{file_path}")


In [22]:
import plotly.graph_objects as go

traces = []
for risk_level, mix_loan_types in combination_dict.items():
    bank_names_list = []
    shpizer_payback_rate = []
    keren_shava_playback_rate = []
    
    # Iterate through unique bank names
    for bank_name in unique_bank_names: 
        bank_names_list.append(bank_name)
        bank_data = banks_info.loc[banks_info['Bank'] == bank_name]

        # Reset rates for each bank
        total_shpizer_rate = 0
        total_keren_shava_rate = 0
        loan_count = 0  # For counting loan types
        
        for single_loan_type in mix_loan_types:
            interest_rate = bank_data.loc[bank_data["loan_type"] == single_loan_type]["Interest Rate"].values[0]

            dict_shpizer = calc_total_payback_and_payback_rate_per_type(
                loan_type=single_loan_type, 
                mortgage_amount_nis=mortgage_amount_nis,
                years=num_of_years, 
                annual_interest_rate=interest_rate, 
                payback_method="shpizer"
            )

            dict_keren_shava = calc_total_payback_and_payback_rate_per_type(
                loan_type=single_loan_type, 
                mortgage_amount_nis=mortgage_amount_nis,               
                years=num_of_years, 
                annual_interest_rate=interest_rate, 
                payback_method="keren_shava"
            )

            # Accumulate rates for weighted average calculations
            if risk_level == "middle_risk":
                total_shpizer_rate += dict_shpizer["payback_rate"] * 0.333  # Weight for middle risk
                total_keren_shava_rate += dict_keren_shava["payback_rate"] * 0.333  # Weight for middle risk
                loan_count += 1  # Count loan types
            else:  # For low risk, just append
                shpizer_payback_rate.append(dict_shpizer["payback_rate"])
                keren_shava_playback_rate.append(dict_keren_shava["payback_rate"])

        # Calculate average rates for middle risk if any loan types were counted
        if loan_count > 0:
            shpizer_payback_rate.append(total_shpizer_rate)  # Add the weighted average for middle risk
            keren_shava_playback_rate.append(total_keren_shava_rate)  # Add the weighted average for middle risk

    # Create a trace for the current risk level
    trace = go.Scatter(
        x=bank_names_list,    
        y=shpizer_payback_rate,         
        mode='lines+markers',    
        name=f'Payback Rate - {risk_level} - Shpizer Method'  # Ensure unique trace names
    )
    traces.append(trace)
    
    trace = go.Scatter(
        x=bank_names_list,    
        y=keren_shava_playback_rate,         
        mode='lines+markers',    
        name=f'Payback Rate - {risk_level} - Keren-Shava Method'  # Ensure unique trace names
    )
    traces.append(trace)

# Create the layout for the figure
layout = go.Layout(
    title="Payback Rate by Bank",
    xaxis_title="Bank",
    yaxis_title="Payback Rate",
    hovermode='closest',
    showlegend=True,  # Explicitly show the legend
    legend=dict(
        orientation='v',  # Vertical orientation
        x=0,  # Position on the left
        xanchor='left',  # Anchor to the left
        y=1,  # Position at the top
        yanchor='top'  # Anchor to the top
    )
)

# Create the figure once after processing all banks
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()

# Optionally, save the figure to an HTML file
# fig.write_html("figure.html")
# Open the HTML file in a browser
# import webbrowser
# import os
# file_path = os.path.abspath("figure.html")
# webbrowser.open(f"file://{file_path}")


TypeError: 'NoneType' object is not subscriptable